In [1]:
import os 
from typing import List, Dict, Any
import pandas as pd


In [2]:
from langchain_core.documents import Document
from langchain_text_splitters import(RecursiveCharacterTextSplitter, CharacterTextSplitter, TextSplitter)

print("Set up completed!")


/Users/yashrepal/My_Workspace/RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Set up completed!


In [4]:
doc = Document(
    page_content= "hey, this is the page content section of document, which will be embedded and searched",
    metadata= {
        "source": "exm.txt",
        "author": "sfgjdh"
    }
)
doc

Document(metadata={'source': 'exm.txt', 'author': 'sfgjdh'}, page_content='hey, this is the page content section of document, which will be embedded and searched')

### Creating simple text file from data


In [8]:
os.makedirs("data/text_fils", exist_ok=True)

#### Defining content

In [13]:
sample_text={
    "data/text_fils/problem_stmt.txt": """You have to build an Agentic AI System that transforms a traditional search-and-
click pharmacy into an autonomous ecosystem. The system should behave like
an expert pharmacist, understand natural voice/text conversations, predict medicine
refill needs, enforce safety and prescription rules, and autonomously execute
backend tasks such as inventory updates and procurement—with minimal human
intervention.""",

 "data/text_fils/req.txt": """Core Functional Requirements
Your system will be judged on its ability to handle these Agentic Capabilities:
 Conversational Ordering: A natural interface (text-based and voice-based) that
can extract medicine names, dosages, and quantities from messy, human like
dialogue.
 Safety & Policy Enforcement: The system must use the provided Medicine
Excel Sheet as its "Source of Truth." It must autonomously decide if an order can
proceed based on stock levels and "Prescription Required" flags.
 Predictive Intelligence: The system must be "Proactive." Based on the
Mock Customer History, it should identify which users are running out of
medicine and initiate a refill conversation or alert
 Real-world Action (Tool Use): The system shouldn't just "talk." It must execute
actions such as updating your mock database, triggering webhooks (mock
webhooks or n8n/Zapier), or sending order confirmations via channels like email,
WhatsApp
Data & Environment
A. Data Assets
1. 2. Medicine Master Data (Excel/CSV): Contains a list of medicines, current
stock levels, unit types, and "Prescription Required" flags.
Consumer Order History (Excel/CSV): Contains historical data for a set of
patients, including what they bought, the date of purchase, and the dosage
frequency.
B. Mock Backend
 No CMS provided: You must build a mock API/Backend (FastAPI, Node.js, Supabase,
etc.) to host the provided data.
 Your agents must interact with this backend to read inventory, check user history,
and write new orders.
C. Observability (Mandatory)
Since this is an agentic system, traceability is non-negotiable.
 You must integrate an observability platform (e.g., Langfuse, LangSmith).
 The judges must be able to see the Chain of Thought (CoT): How Agent A talked to
Agent B, and why Agent B approved/rejected a request.
 Submissions without a live Trace Log link will be disqualified.
D. Minimal UI
 A simple UI is enough but can be enhanced to show the features
 It must include a Chat-based interface and Voice capability is a high-value priority
 A small "Admin View" showing the Mock Inventory levels and any "Proactive Refill"
alerts the system has generated.
E. Workflow Automation
 Show a real-world When an order is finalized, your agent must trigger a warehouse
fulfillment request via a mock Webhook or API. (Optional Bonus: Automate confirmation
emails or WhatsApp messages).
E. Submission Checklist
 GitHub Repo: Clean code and a README for your Mock APIs.
 Observability Public Link: Access to view your agentic trace logs.

"""
}

for filepath,content in sample_text.items():
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content)

#### Reading text file that has beeen created

In [26]:
from langchain_community.document_loaders import TextLoader

loader=TextLoader("data/text_fils/problem_stmt.txt", encoding='utf-8')

loader

documents=loader.load()
print(documents)
print("Metadata ===",documents[0].metadata)
print("Page content ===", documents[0].page_content)


[Document(metadata={'source': 'data/text_fils/problem_stmt.txt'}, page_content='You have to build an Agentic AI System that transforms a traditional search-and-\nclick pharmacy into an autonomous ecosystem. The system should behave like\nan expert pharmacist, understand natural voice/text conversations, predict medicine\nrefill needs, enforce safety and prescription rules, and autonomously execute\nbackend tasks such as inventory updates and procurement—with minimal human\nintervention.')]
Metadata === {'source': 'data/text_fils/problem_stmt.txt'}
Page content === You have to build an Agentic AI System that transforms a traditional search-and-
click pharmacy into an autonomous ecosystem. The system should behave like
an expert pharmacist, understand natural voice/text conversations, predict medicine
refill needs, enforce safety and prescription rules, and autonomously execute
backend tasks such as inventory updates and procurement—with minimal human
intervention.


In [33]:
from langchain_community.document_loaders import DirectoryLoader

dir_load=DirectoryLoader(
    "data/text_fils",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding":"utf-8"},
    show_progress=True
)


documets=dir_load.load()
print(documets[1].metadata)

100%|██████████| 2/2 [00:00<00:00, 1204.74it/s]

{'source': 'data/text_fils/req.txt'}


#### Text splitting

In [35]:
from langchain_text_splitters import(
    RecursiveCharacterTextSplitter, TextSplitter, CharacterTextSplitter
)
documents

[Document(metadata={'source': 'data/text_fils/problem_stmt.txt'}, page_content='You have to build an Agentic AI System that transforms a traditional search-and-\nclick pharmacy into an autonomous ecosystem. The system should behave like\nan expert pharmacist, understand natural voice/text conversations, predict medicine\nrefill needs, enforce safety and prescription rules, and autonomously execute\nbackend tasks such as inventory updates and procurement—with minimal human\nintervention.')]

In [38]:
text=documents[0].page_content
text

'You have to build an Agentic AI System that transforms a traditional search-and-\nclick pharmacy into an autonomous ecosystem. The system should behave like\nan expert pharmacist, understand natural voice/text conversations, predict medicine\nrefill needs, enforce safety and prescription rules, and autonomously execute\nbackend tasks such as inventory updates and procurement—with minimal human\nintervention.'

In [54]:
# character text splitter
character_splitter=CharacterTextSplitter(
    separator="\n",
    chunk_size=80,
    chunk_overlap=20,
    length_function=len
)

char_chunks=character_splitter.split_text(text)
char_chunks

print(len(char_chunks))
# print(char_chunks[0])

Created a chunk of size 83, which is longer than the specified 80


6


In [53]:
# Recursive character text splitter

recursive_text_splitter=RecursiveCharacterTextSplitter(
    separators=" ",
    chunk_size=80,
    chunk_overlap=30,
    length_function=len
)

recursive_char_chunks=recursive_text_splitter.split_text(text)
recursive_char_chunks
print(len(recursive_char_chunks))
print(recursive_char_chunks)

8
['You have to build an Agentic AI System that transforms a traditional', 'that transforms a traditional search-and-\nclick pharmacy into an autonomous', 'pharmacy into an autonomous ecosystem. The system should behave like\nan expert', 'should behave like\nan expert pharmacist, understand natural voice/text', 'understand natural voice/text conversations, predict medicine\nrefill needs,', 'medicine\nrefill needs, enforce safety and prescription rules, and autonomously', 'rules, and autonomously execute\nbackend tasks such as inventory updates and', 'such as inventory updates and procurement—with minimal human\nintervention.']
